# Обучение парсингу

## Requests

🧩 requests — для скачивания страниц  
Библиотека requests нужна, чтобы отправлять HTTP-запросы и получать данные из интернета.  
Что она делает:  
скачивает HTML-страницы (GET)  
отправляет формы (POST)  
добавляет куки, заголовки, токены  
работает с API


In [5]:
#Пример парсинга ключевой ставки со страницы ЦБ:

import requests

url = "https://www.cbr.ru/hd_base/KeyRate/"
params = {
    "UniDbQuery.Posted": "True",
    "UniDbQuery.From": "07.05.2025",   # начало периода
    "UniDbQuery.To": "14.11.2025",     # конец периода
}

r = requests.get(url, params=params)
html = r.text

html[:300]

'\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n    \r\n\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\r\n<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\r\n<meta name="format-detection" cont'

# 🍜 BeautifulSoup — для парсинга HTML
Классическая библиотека для разбора HTML-кода и «доставания» нужных кусочков.  
Что она делает:  
находит теги (div, span, a…)  
ищет по классам, id, атрибутам  
достаёт текст, ссылки, таблицы  
очищает HTML  

🎯 В парсинге они работают вместе  
requests скачивает страницу  
BeautifulSoup её анализирует и достаёт данные

In [13]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")
table = soup.find("table")


In [14]:
type(table)

bs4.element.Tag

In [17]:
#Найти вложенные элементы

rows = table.find_all("tr")
# rows

In [24]:
#🔹 Перебирать строки таблицы

k = 0

for tr in table.find_all("tr"):
    cells = tr.find_all(["td", "th"])
    print(cells)
    k += 1
    if k == 10:
        break

[<th>Дата</th>, <th>Ставка</th>]
[<td>14.11.2025</td>, <td>16,50</td>]
[<td>13.11.2025</td>, <td>16,50</td>]
[<td>12.11.2025</td>, <td>16,50</td>]
[<td>11.11.2025</td>, <td>16,50</td>]
[<td>10.11.2025</td>, <td>16,50</td>]
[<td>07.11.2025</td>, <td>16,50</td>]
[<td>06.11.2025</td>, <td>16,50</td>]
[<td>05.11.2025</td>, <td>16,50</td>]
[<td>03.11.2025</td>, <td>16,50</td>]


In [28]:
# Полная рабочая схема:

import requests
from bs4 import BeautifulSoup

url = "https://www.cbr.ru/hd_base/KeyRate/"
params = {
    "UniDbQuery.Posted": "True",
    "UniDbQuery.From": "07.05.2025",
    "UniDbQuery.To": "14.11.2025",
}

r = requests.get(url, params=params)
soup = BeautifulSoup(r.text, "html.parser")

table = soup.find("table")  # это твой bs4.element.Tag

rows = table.find_all("tr")  # список тегов <tr>

data = []

for tr in rows[1:]:  # пропускаем заголовок
    tds = tr.find_all("td")   # все <td> в текущей строке
    
    # если не 2 колонки — пропускаем
    if len(tds) != 2:
        continue

    date = tds[0].get_text(strip=True)
    rate = tds[1].get_text(strip=True).replace(",", ".")
    data.append((date, float(rate)))

for row in data[:5]:
    print(row)


('14.11.2025', 16.5)
('13.11.2025', 16.5)
('12.11.2025', 16.5)
('11.11.2025', 16.5)
('10.11.2025', 16.5)
